In [37]:
from compiler import (parse_toy, print_op, optimise_toy, lower_from_toy, 
                                  optimise_vir, lower_to_riscv, emulate_riscv)

from riscv.emulator_iop import run_riscv, print_riscv_ssa

### Notebook 2

example = """
def main() {
  var a<2, 3> = [[1, 2, 3], [4, 5, 6]];
  var b<6> = [1, 2, 3, 4, 5, 6];
  var c<2, 3> = b;
  var d = a + c;
  print(d);
}
"""

toy_0 = parse_toy(example)
print_op(toy_0)
print()

# mention the unnecessary

"builtin.module"() ({
  "toy.func"() ({
    %0 = "toy.constant"() {"value" = dense<[[1, 2, 3], [4, 5, 6]]> : tensor<2x3xi32>} : () -> tensor<2x3xi32>
    %1 = "toy.reshape"(%0) : (tensor<2x3xi32>) -> tensor<2x3xi32>
    %2 = "toy.constant"() {"value" = dense<[1, 2, 3, 4, 5, 6]> : tensor<6xi32>} : () -> tensor<6xi32>
    %3 = "toy.reshape"(%2) : (tensor<6xi32>) -> tensor<6xi32>
    %4 = "toy.reshape"(%3) : (tensor<6xi32>) -> tensor<2x3xi32>
    %5 = "toy.add"(%1, %4) : (tensor<2x3xi32>, tensor<2x3xi32>) -> tensor<2x3xi32>
    "toy.print"(%5) : (tensor<2x3xi32>) -> ()
    "toy.return"() : () -> ()
  }) {"sym_name" = "main", "function_type" = () -> ()} : () -> ()
}) : () -> ()



In [ ]:
print_op(toy_0)
print()

# mention the unnecessary reshapes

In [38]:
### Chapter 3

toy_1 = optimise_toy(toy_0)
print_op(toy_1)
print()

### Contents

# Reshape of a constant can be done by changing the constant
# 

"builtin.module"() ({
  "toy.func"() ({
    %0 = "toy.constant"() {"value" = dense<[[1, 2, 3], [4, 5, 6]]> : tensor<2x3xi32>} : () -> tensor<2x3xi32>
    %1 = "toy.constant"() {"value" = dense<[[1, 2, 3], [4, 5, 6]]> : tensor<2x3xi32>} : () -> tensor<2x3xi32>
    %2 = "toy.add"(%0, %1) : (tensor<2x3xi32>, tensor<2x3xi32>) -> tensor<2x3xi32>
    "toy.print"(%2) : (tensor<2x3xi32>) -> ()
    "toy.return"() : () -> ()
  }) {"sym_name" = "main", "function_type" = () -> ()} : () -> ()
}) : () -> ()



In [27]:
# Chapter 4

vir_0 = lower_from_toy(toy_1)
print_op(vir_0)
print()

### Contents

# Lowering implementation


"builtin.module"() ({
  "toy.func"() ({
    %0 = "toy.vector.constant"() {"data" = [#int<2>, #int<3>], "label" = "tensor_shape"} : () -> #riscv_ssa.reg
    %1 = "toy.vector.constant"() {"data" = [#int<1>, #int<2>, #int<3>, #int<4>, #int<5>, #int<6>], "label" = "tensor_data"} : () -> #riscv_ssa.reg
    %2 = "toy.tensor.make"(%0, %1) : (#riscv_ssa.reg, #riscv_ssa.reg) -> #riscv_ssa.reg
    "toy.print"(%2) : (#riscv_ssa.reg) -> ()
    "toy.return"() : () -> ()
  }) {"sym_name" = "main", "function_type" = () -> ()} : () -> ()
}) : () -> ()



In [28]:
# Chapter 5

vir_1 = optimise_vir(vir_0)
print_op(vir_1)
print()

"builtin.module"() ({
  "toy.func"() ({
    %0 = "toy.vector.constant"() {"data" = [#int<2>, #int<3>], "label" = "tensor_shape"} : () -> #riscv_ssa.reg
    %1 = "toy.vector.constant"() {"data" = [#int<1>, #int<2>, #int<3>, #int<4>, #int<5>, #int<6>], "label" = "tensor_data"} : () -> #riscv_ssa.reg
    %2 = "toy.tensor.make"(%0, %1) : (#riscv_ssa.reg, #riscv_ssa.reg) -> #riscv_ssa.reg
    "toy.print"(%2) : (#riscv_ssa.reg) -> ()
    "toy.return"() : () -> ()
  }) {"sym_name" = "main", "function_type" = () -> ()} : () -> ()
}) : () -> ()



In [29]:
# Chapter 6

riscv_0 = lower_to_riscv(vir_1)
print_op(riscv_0)
print()

"builtin.module"() ({
  "riscv.section"() ({
    "riscv_ssa.label"() {"label" = #riscv.label<heap>} : () -> ()
    "riscv_ssa.directive"() {"directive" = ".space", "value" = "1024"} : () -> ()
  }) {"directive" = ".bss"} : () -> ()
  "riscv.section"() ({
    "riscv_ssa.label"() {"label" = #riscv.label<main.tensor_shape.5>} : () -> ()
    "riscv_ssa.directive"() {"directive" = ".word", "value" = "0x2, 0x2, 0x3"} : () -> ()
    "riscv_ssa.label"() {"label" = #riscv.label<main.tensor_data.5>} : () -> ()
    "riscv_ssa.directive"() {"directive" = ".word", "value" = "0x6, 0x1, 0x2, 0x3, 0x4, 0x5, 0x6"} : () -> ()
  }) {"directive" = ".data"} : () -> ()
  "riscv.section"() ({
    "riscv_ssa.func"() ({
      %0 = "riscv_ssa.li"() {"immediate" = #riscv.label<heap>} : () -> #riscv_ssa.reg
      %1 = "riscv_ssa.li"() {"immediate" = #riscv.label<main.tensor_shape.5>} : () -> #riscv_ssa.reg
      %2 = "riscv_ssa.li"() {"immediate" = #riscv.label<main.tensor_data.5>} : () -> #riscv_ssa.reg
      %3

In [30]:
code = print_riscv_ssa(riscv_0)
print(code)
print()

.bss 
heap:
.space 1024
.data 
main.tensor_shape.5:
.word 0x2, 0x2, 0x3
main.tensor_data.5:
.word 0x6, 0x1, 0x2, 0x3, 0x4, 0x5, 0x6
.text 
main:
	li	%0, heap
	li	%1, main.tensor_shape.5
	li	%2, main.tensor_data.5
	li	%3, 2
	li	%4, 4
	mul	%5, %3, %4		# Alloc count bytes
	lw	%6, %0, 0		# Old heap count
	add	%7, %6, %5		# New heap count
	sw	%7, %0, 0		# Update heap
	addi	%8, %0, 4		# Heap storage start
	add	%9, %8, %6		# Allocated memory
	sw	%1, %9, 0		# Set tensor shape
	lw	%10, %9, 0
	sw	%2, %9, 4		# Set tensor data
	toy.print	%9
	li	a7, 93
	scall




In [31]:
emulate_riscv(code)

[CPU] Started running from example.asm:.text at heap (0x100) + 0x428
Program(name=example.asm,sections=set(),base=['.bss', '.data', '.text'])
[[1, 2, 3], [4, 5, 6]]
[CPU] Program exited with code 0
